# Generatory oraz iteratory

## Generatory

### yield, next

**Funkcja**

In [1]:
def calculate_squares_func(x):
    output = []
    for i in range(x):
        output.append(i ** 2)
    return output

In [3]:
calculate_squares_func(10)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [4]:
%%timeit -n 1000 -r 1000

calculate_squares_func(10)

The slowest run took 5.14 times longer than the fastest. This could mean that an intermediate result is being cached.
1.21 µs ± 429 ns per loop (mean ± std. dev. of 1000 runs, 1,000 loops each)


In [7]:
func_output = calculate_squares_func(1000)  # 100, 1000

In [8]:
import sys

sys.getsizeof(func_output)

8856

**Generator**

In [10]:
def calculate_squares_gen(x):
    for i in range(x):
        yield i ** 2

In [21]:
generator_object = calculate_squares_gen(10)
generator_object

<generator object calculate_squares_gen at 0x7a99b34a6740>

In [23]:
next(generator_object)

StopIteration: 

In [22]:
list(generator_object)

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [24]:
for i in calculate_squares_gen(10):
    print(i)

0
1
4
9
16
25
36
49
64
81


In [26]:
%%timeit -n 1000  -r 1000
calculate_squares_gen(100000000000)

The slowest run took 6.18 times longer than the fastest. This could mean that an intermediate result is being cached.
477 ns ± 153 ns per loop (mean ± std. dev. of 1000 runs, 1,000 loops each)


In [29]:
gen_output = calculate_squares_gen(10000000)  # 100, 1000

In [30]:
sys.getsizeof(gen_output)

208

- lazy evaluation
- oszczędza pamięć
- jest szybszy
- nie mylić `generator` i `generator_object`

---
---

In [31]:
def numbers_generator():
    numbers = [1, 2, 3, 4, 5]
    for number in numbers:
        yield number

In [33]:
numbers = numbers_generator()
numbers

<generator object numbers_generator at 0x7a99b34a5be0>

In [34]:
for n in numbers:
    print(n)

1
2
3
4
5


In [35]:
next(numbers)

StopIteration: 

> **ZADANIA**

### `__iter__`, `__next__`, generator expression, generator nieskończony

In [36]:
numbers = numbers_generator()

In [37]:
numbers

<generator object numbers_generator at 0x7a99b34a4930>

In [38]:
next(numbers)

1

In [40]:
numbers.__next__()

3

In [ ]:
next(numbers)  # -> numbers.__next__()

In [42]:
[1, 2, 3].__iter__()

In [51]:
numbers.__iter__().__iter__().__iter__().__iter__()

<generator object numbers_generator at 0x7a99b34a4930>

In [ ]:
numbers.__iter__()  # iter()

In [ ]:
numbers.__next__()  # next()

---

In [53]:
[i for i in range(4)]

[0, 1, 2, 3]

In [58]:
def generator(x):
    for i in range(x):
        yield i

list(generator(4))

[0, 1, 2, 3]

In [60]:
(i for i in range(4))

<generator object <genexpr> at 0x7a99b171db10>

In [59]:
list((i for i in range(4)))

[0, 1, 2, 3]

---

In [61]:
def infinite_generator():
    """
    use case: ID numbers generator
    """
    i = 0
    while True:
        yield i
        i += 1

In [62]:
gen = infinite_generator()

In [120]:
next(gen)

57

> **ZADANIA**

## Iteratory

Główna idea - tworzenie własnych klas, których instancje będą iterablami

Iteratory są bardziej efektywne pod względem pamięci, ponieważ nie przechowują wszystkich wartości naraz a dostają się do nich po kolei.

In [ ]:
nums = [1, 2, 3]

In [ ]:
dir(nums)

In [ ]:
nums.__iter__()  # lista zamieniona na iterator

# iterator to obiekt po którym można iterować a on pamięta na którym elemenencie akutalnie jesteśmy

In [ ]:
next(nums)  # na liście nie można wywoływać next

In [ ]:
iter_nums = nums.__iter__()

iter_nums = iter(nums)  # alternatywny zapis

In [ ]:
iter_nums

In [ ]:
next(iter_nums)

Jak działa pętla for po liście - zamiana listy na iterator, wykonywanie pętli tak długo aż będzie StopIteration error

In [ ]:
nums

In [ ]:
for num in nums:
    print(num)

In [ ]:
# tak działa pętla for
nums_iter = iter(nums)
while True:
    try:
        num = next(nums_iter)
        print(num)
    except StopIteration:
        break

---

Napiszmy własny iterator

(uwaga - `range()` nie jest iteratorem ponieważ nie ma `__next__()`)

In [ ]:
class MyRange:
    def __init__(self, start, end):
        self.value = start
        self.end = end
        
    def __iter__(self):
        # jak coś jest iteratorem, to jego __iter__ zwraca ten obiekt
        # iterując po X iterujemy tak naprawdę po X.__iter__()
        return self

    def __next__(self):
        if self.value >= self.end:
            raise StopIteration
        
        current = self.value
        self.value += 1
        return current

In [ ]:
myrange = MyRange(1, 10)

In [ ]:
myrange

In [ ]:
for num in myrange:
    print(num)

Generator objects (zwracane przez generatory) też są iteratorami, ale nie musimy wprost definiować im `__next__` ani `__iter__` ponieważ nie tworzymy klasy.

In [ ]:
def my_range(start, end):
    current = start
    while current < end:
        yield current
        current += 1

In [ ]:
for i in my_range(1, 10):
    print(i)

Przykład:

In [ ]:
class FibonacciIterator:
    def __init__(self, limit):
        self.limit = limit
        self.a, self.b = 1, 1
        self.count = 0

    def __iter__(self):
        return self

    def __next__(self):
        if self.count >= self.limit:
            raise StopIteration

        result = self.a
        self.a, self.b = self.b, self.a + self.b
        self.count += 1
        return result

# Przykład użycia
fibonacci_iter = FibonacciIterator(10)

for num in fibonacci_iter:
    print(num)


Nieskończony iterator

In [ ]:
class MyRange:
    def __init__(self, start):
        self.value = start
        
    def __iter__(self):
        return self
    
    def __next__(self):
        current = self.value
        self.value += 1
        return current

In [ ]:
myr = MyRange(4)

In [ ]:
next(myr)

> **ZADANIA**